# Everything starts here

In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import utils
from model import HerniaModel

## 1. Split train / validation

In [2]:
# do the split only if it has not already been done
if not os.path.exists(utils.dfs_path + '/training_finetuning.pkl') or not os.path.exists(utils.dfs_path + '/validation_finetuning.pkl'):
    # load all train videos (labelled videos)
    all_train_videos = utils.get_train_test_video_names()['train']
    all_train_labels = pd.read_pickle(utils.labels_path)

    # define split
    split = 0.8
    np.random.seed(69)
    train_videos = np.array(all_train_videos)[np.random.choice(len(all_train_videos), int(0.8 * len(all_train_videos)), replace=False)]
    validation_videos = np.setdiff1d(all_train_videos, train_videos, assume_unique=False)
    train_videos.sort()
    validation_videos.sort()

    # create two subdataframes for training and validation
    training_df = all_train_labels.loc[all_train_labels['videoname'].isin(train_videos)]
    validation_df = all_train_labels.loc[all_train_labels['videoname'].isin(validation_videos)]

    training_df.to_pickle(utils.dfs_path + '/training_finetuning.pkl')
    validation_df.to_pickle(utils.dfs_path + '/validation_finetuning.pkl')

## 2. Model parameters setting

In [2]:
# rebuild model
model_parameters = ('mobilenet_v2_lstm', utils.num_classes, True, 1, False, 16, False)

## 3. CNN (backbone) finetuning

### 3.0 Reload model

In [ ]:
model = HerniaModel(*model_parameters).to(utils.device)

In [ ]:
# skip lstm
model.skip_lstm = True

### 3.1 Set training hyperparameters

In [7]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [8]:
# training parameters
LEARNING_RATE = 0.002
EPOCHS = 1
BATCH_SIZE = 128
MOMENTUM = 0.9
GAMMA = 0.5
STEP_SIZE = 1

In [10]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [11]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_finetuning.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [12]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['training', 'validation']}

### 3.2 Train

In [14]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/1
----------
164841/164841: [===============================>] - ETA 0.9ssss
training Loss: 0.4376 Acc: 0.8528
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.6508 Acc: 0.8003
Training complete in 19m 6s
Best val Acc: 0.800343


## 4. LSTM training

### 4.0 Reload model

In [20]:
# rebuild model
model = HerniaModel(*model_parameters).to(utils.device)

In [21]:
# do not skip lstm and freeze backbone
model.skip_lstm = False
model.freeze_backbone()

In [22]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

### 4.1 Pad with black images for training of LSTM

In order to process consecutive frames by batch using our dataloaders, each batch loaded should only be composed of frames of a single video. A trick consists of padding each video with blank images at the end.

In [24]:
BATCH_SIZE = 32

if not os.path.exists(utils.dfs_path + '/training_lstm.pkl') or not os.path.exists(utils.dfs_path + '/validation_lstm.pkl'):
    for phase in ['training', 'validation']:
        df = pd.read_pickle(utils.dfs_path + '/' + phase + '_finetuning.pkl')
        for videoname in df['videoname'].unique():
            # pad with black images at the end of each video
            # so as to only have number of frames multiple of BATCH_SIZE
            # we can then process through the LSTM by batch without shuffling
            num_frames = df[df.videoname == videoname].shape[0]
            num_rows_to_add = (BATCH_SIZE - (num_frames % BATCH_SIZE)) % BATCH_SIZE
            template_white_row = {'videoname': videoname, 'frame': 10000, 'label': -1}
            white_rows_to_add = pd.DataFrame([template_white_row for _ in range(num_rows_to_add)])
            df = pd.concat([df, white_rows_to_add], ignore_index=True)
        df['sort'] = df['videoname'].str[-12].astype(int) * 10000 + df['videoname'].str[-3:].astype(int)
        # sort rows
        df.sort_values(['sort', 'frame'],inplace=True, ascending=True)
        df.reset_index(inplace=True)
        df = df.drop(['sort', 'index'], axis=1)
        # shuffle batches
        index_list = np.array(df.index)
        np.random.shuffle(np.reshape(index_list, (-1, BATCH_SIZE)))
        shuffled_df = df.loc[index_list, :]
        shuffled_df.reset_index(inplace=True)
        shuffled_df = shuffled_df.drop(['index'], axis=1)
        # save df
        shuffled_df.to_pickle(utils.dfs_path + '/' + phase + '_lstm.pkl')

### 4.2 Set training hyperparameters

In [25]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [26]:
# training parameters
LEARNING_RATE = 0.005
EPOCHS = 20
MOMENTUM = 0.9
GAMMA = 0.75
STEP_SIZE = 2

In [27]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [28]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_lstm.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [29]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=False) for x in ['training', 'validation']}

### 4.3 Train

In [30]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/20
----------
165728/165728: [===============================>] - ETA 0.1sss
training Loss: 2.0076 Acc: 0.2466
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 1.7738 Acc: 0.3563
Epoch 2/20
----------
165728/165728: [===============================>] - ETA 0.1sss
training Loss: 1.9323 Acc: 0.2875
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 1.6177 Acc: 0.3998
Epoch 3/20
----------
165728/165728: [===============================>] - ETA 0.1sss
training Loss: 1.8776 Acc: 0.3154
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 1.4467 Acc: 0.6072
Epoch 4/20
----------
165728/165728: [===============================>] - ETA 0.1sss
training Loss: 1.8524 Acc: 0.3217
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 1.5167 Acc: 0.4220
Epoch 5/20
----------
165728/165728: [===============================>] - ETA 0.1sss
training Loss: 1.7930 Acc: 0.3616
50464/50464: [===

## 5. Whole network finetuning

### 5.0 Reload model

In [39]:
# rebuild model
model = HerniaModel(*model_parameters).to(utils.device)

In [40]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

### 5.1 Set training hyperparameters

In [41]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [42]:
# training parameters
LEARNING_RATE = 0.0008
EPOCHS = 3
BATCH_SIZE = 32
MOMENTUM = 0.9
GAMMA = 0.3
STEP_SIZE = 1

In [43]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [44]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_lstm.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [45]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=False) for x in ['training', 'validation']}

### 5.2 Train

In [46]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/3
----------
165728/165728: [===============================>] - ETA 0.2ssss
training Loss: 1.8947 Acc: 0.3164
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 2.0512 Acc: 0.2456
Epoch 2/3
----------
165728/165728: [===============================>] - ETA 0.2sss
training Loss: 1.4319 Acc: 0.5317
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 1.8941 Acc: 0.2812
Epoch 3/3
----------
  3744/165728: [>...............................] - ETA 947.6s

KeyboardInterrupt: 

## 6. Make Kaggle prediction

In [ ]:
# create and save testing df
utils.save_testing_df()

In [ ]:
BATCH_SIZE = 128

if not os.path.exists(utils.dfs_path + '/testing_lstm.pkl'):
    for phase in ['testing']:
        df = pd.read_pickle(utils.dfs_path + '/testing.pkl')
        for videoname in df['videoname'].unique():
            # pad with blank images at the end of each video
            # so as to only have number of frames multiple of BATCH_SIZE
            # we can then process through the LSTM by batch without shuffling
            num_frames = df[df.videoname == videoname].shape[0]
            num_rows_to_add = (BATCH_SIZE - (num_frames % BATCH_SIZE)) % BATCH_SIZE
            template_white_row = {'videoname': videoname, 'frame': 10000}
            white_rows_to_add = pd.DataFrame([template_white_row for _ in range(num_rows_to_add)])
            df = pd.concat([df, white_rows_to_add], ignore_index=True)
        df['sort'] = df['videoname'].str[-12].astype(int) * 10000 + df['videoname'].str[-3:].astype(int)
        # sort rows
        df.sort_values(['sort', 'frame'],inplace=True, ascending=True)
        df = df.drop('sort', axis=1)
        # save df
        df.to_pickle(utils.dfs_path + '/' + phase + '_lstm.pkl')

In [ ]:
# rebuild model
model = HerniaModel(model_name = 'mobilenet_v2_lstm', 
                    num_classes = utils.num_classes, 
                    pretrained = True, 
                    num_layers_lstm = 2,
                    bidirectional = False,
                    hidden_size_lstm = 32,
                    skip_lstm = False).to(utils.device)

In [ ]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

In [ ]:
# data augmentation and normalization for training
testing_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
utils.predict_kaggle(model = model, 
                    model_name = model.model_name, # name of the model from which to load the weights within weights/
                    transform = testing_transforms, 
                    predictions_name = model.model_name,
                    batch_size = BATCH_SIZE) # name of the csv file to which the predictions are saved within predictions/

## Bonus. Evaluate model

In [19]:
model = HerniaModel(*model_parameters).to(utils.device)

In [20]:
# skip lstm
model.skip_lstm = False

In [21]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

In [22]:
BATCH_SIZE = 32

# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

In [23]:
# just normalization for validation
data_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [24]:
# create pytorch datasets
dataset = utils.HernitiaDataset(utils.dfs_path + '/' + 'validation' + '_lstm.pkl', data_transform)

In [25]:
# instantiate data loaders
dataloader = utils.DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)

In [26]:
utils.evaluate_model(model, dataloader, criterion)

50464/50464: [===============================>] - ETA 0.1sss
Loss: 1.4467 Acc: 0.6072
